# Graph Class and Functions

In [1]:
import numpy as np
import pandas as pd

class Graph:
    
    def __init__(self):
        self.nodes = {}
        self.capacities = {}
    
    def add_weights(self, label, edges):
        '''
        label - The label of the node.
        
        edges - A dictionary where the keys are the nodes to which the outgoing 
                arcs are going and the keys are the arc weights.
        '''
        self.nodes[label] = edges
        
    def add_bounds(self, label, edges):
        '''
        label - The label of the node.
        
        edges - A dictionary where the keys are the nodes to which the outgoing 
                arcs are going and the keys are the arc upper bound capacities.
        '''
        self.capacities[label] = edges
        
    
    def _reverse_edges(self, edges, negate_weights = False, init_zeros = False):
        reversed_edges = {}
        for node in edges:
            reversed_edges[node] = {}

        for node in edges:
            for path in edges[node]:
                if negate_weights:
                    reversed_edges[path][node] = -edges[node][path]
                elif init_zeros:
                    reversed_edges[path][node] = 0
                else:
                    reversed_edges[path][node] = edges[node][path]
                
        return reversed_edges
    
    
    def BellmanFord(self, begin, end):
        nodes = self.nodes.keys()
        node_dict = self._reverse_edges(self.nodes)

        dst_table = pd.DataFrame(index = nodes)
        path_table = pd.DataFrame(index = nodes)

        dst_table['init dst'] = np.where(dst_table.index == begin, 0, np.inf)
        path_table['init path'] = 'unk'
        
        stop = False
        iteration = 1
        while not stop:
            dst_table['dst_' + str(iteration)] = dst_table.iloc[:,-1]
            path_table['path_' + str(iteration)] = path_table.iloc[:,-1]
            
            for node in nodes:
                current = dst_table.loc[node][-1]

                for edge in node_dict[node].keys():
                    potential = dst_table.loc[edge][-2] + node_dict[node][edge]
                    if potential < current:
                        dst_table.loc[node][-1] = potential
                        path_table.loc[node][-1] = edge
        
            # make distance to negative cost cycle nodes infinity
            if iteration >= len(nodes)+2 and not dst_table.iloc[:,-1].equals(dst_table.iloc[:,-2]):
                for cycle_node in dst_table.index[dst_table.iloc[:,-1] != dst_table.iloc[:,-2]]:
                    dst_table.loc[cycle_node][-1] = np.inf

            
            if dst_table.iloc[:,-1].equals(dst_table.iloc[:,-2]):
                stop = True
                
            else:
                iteration += 1
        
        dst_table.where(dst_table != np.inf, 'inf', inplace=True)
        
        path_table.loc[begin] = begin
        
        
        full_table = pd.DataFrame()
        dst_cols = dst_table.columns
        path_cols = path_table.columns
        
        for i in range(dst_table.shape[1]):
            full_table[dst_cols[i]] = dst_table[dst_cols[i]]
            full_table[path_cols[i]] = path_table[path_cols[i]]

        return full_table, dst_table, path_table
    
    
    def Dijkstra(self, begin, end, bottleneck = False):
        '''
        Setting "bottleneck" to True will find the S to T path with the smallest maximum edge size. 
        The maximum edge size in the path to each node is what is propagated into dst_table when "
        bottleneck" is True.
        '''
        
        known = []
        unknown = list(self.nodes.keys())
        node_dict = self.nodes
        
        dst_table = pd.DataFrame({begin:np.inf}, index = unknown)
        dst_table.loc[begin, begin] = 0
        
        path_table = pd.DataFrame({begin:'unk'}, index = unknown)
        path_table.loc[begin, begin] = begin
        
        for node in node_dict[begin].keys():
            dst_table.loc[node, begin] = node_dict[begin][node]
            path_table.loc[node, begin] = begin
        
        known.append(begin)
        unknown.remove(begin)
        
        while end in unknown:
            closest = ''
            closest_dst = np.inf
            for node in unknown:
                if dst_table.loc[node][-1] <= closest_dst:
                    closest = node
                    closest_dst = dst_table.loc[node][-1]
            
            known.append(closest)
            unknown.remove(closest)
            
            dst_table[closest] = dst_table.iloc[:,-1]
            path_table[closest] = path_table.iloc[:,-1]
            
            for node in node_dict[closest].keys():
                if node in unknown:
                    if bottleneck:
                        potential = max(dst_table.loc[closest][-1], node_dict[closest][node])
                    else: 
                        potential = dst_table.loc[closest][-1] + node_dict[closest][node]
                        
                    if potential < dst_table.loc[node][-1]:
                        dst_table.loc[node][-1] = potential
                        path_table.loc[node][-1] = closest
                        
        dst_table.where(dst_table != np.inf, 'inf', inplace=True)
        
        full_table = pd.DataFrame()
        dst_cols = dst_table.columns
        path_cols = path_table.columns
        
        for i in range(dst_table.shape[1]):
            full_table[dst_cols[i] + '_dst'] = dst_table[dst_cols[i]]
            full_table[path_cols[i] + '_path'] = path_table[path_cols[i]]
        
        return full_table, dst_table, path_table
    
    
    def shortest_path(self, begin, end, method = 'BellmanFord', verbose = True):
        if method == 'BellmanFord':
            full_table, dst_table, path_table = self.BellmanFord(begin, end)
        else:
            full_table, dst_table, path_table = self.Dijkstra(begin, end)

        if dst_table.loc[end][-1] == 'inf':
            if verbose:
                print('There is no bounded shortest path from ' + begin + ' to ' + end)
            return None
        
        shortest_path = []
        next_in_line = end
        
        while next_in_line != begin:
            shortest_path.insert(0, next_in_line)
            next_in_line = path_table.loc[next_in_line][-1]

        shortest_path.insert(0, begin)
        
        return tuple(shortest_path)
    
    
    def negative_cycles(self, begin, end, verbose = True):
        full_table, dst_table, path_table = self.BellmanFord(begin, end)
        
        if dst_table.iloc[:,-1].dtype != 'O':
            if verbose:
                print('There is no Negative Cost Cycle')
            return None

        cycles = []
        for c_node in dst_table.loc[dst_table.iloc[:,-1] == 'inf'].index:

            if all(c_node not in cycle for cycle in cycles):
                next_in_line = c_node
                cycle = []

                while len(cycle) == len(set(cycle)):
                    cycle.insert(0, next_in_line)
                    next_in_line = path_table.loc[next_in_line][-1]

                cycle = tuple(cycle[:cycle[1:].index(cycle[0]) + 2])
                if set(cycle) not in (set(n_cycle) for n_cycle in cycles) and len(cycle) > 2: 
                    cycles.append(cycle)
            
        if len(cycles) == 0: 
            if verbose:
                print('There is no Negative Cost Cycle')
            return None

        return tuple(cycles)
        

    def flow_augmentation(self, begin, end, cycle_cancelling = False, verbose = True):
        # Initialize edge capacities and weights
        edge_capacities = {}
        edge_weights = {}
        for node in self.nodes:
            # Update capacities
            edge_capacities[node] = {}
            # Update Weights
            edge_weights[node] = {}
            
        for node in self.nodes:
            for edge in self.capacities[node]:
                # Update capacities
                edge_capacities[node][edge] = self.capacities[node][edge]
                edge_capacities[edge][node] = 0
                # Update Weights
                edge_weights[node][edge] = self.nodes[node][edge]
                edge_weights[edge][node] = -self.nodes[node][edge]
                
        # Initial Shortest Path graph
        sp_graph = Graph()
        sp_graph.nodes = self.nodes
        
        if cycle_cancelling: # Get negative cost cycle
            augment_path = sp_graph.negative_cycles(begin, end, verbose = False)
            if augment_path is not None:
                augment_path = augment_path[0]
        else: # Get shortest path
             augment_path = sp_graph.shortest_path(begin, end, verbose = False)
        
        # Iteration
        max_flow = 0
        while augment_path is not None:
            # Find bottleneck capacity on augmenting path
            augment_path_edges = []
            bottleneck = np.inf
            for i in range(len(augment_path)-1):
                augment_path_edges.append((augment_path[i], augment_path[i+1]))
                if edge_capacities[augment_path[i]][augment_path[i+1]] < bottleneck:
                    bottleneck = edge_capacities[augment_path[i]][augment_path[i+1]]
            max_flow += bottleneck
            
            # Augment Flow in edge_capacities dict
            for edge in augment_path_edges:
                edge_capacities[edge[0]][edge[1]] -= bottleneck
                edge_capacities[edge[1]][edge[0]] += bottleneck
                
            # update sp_graph edges
            augmented_graph_edges = {}
            for node in edge_capacities:
                augmented_graph_edges[node] = {}
                
            for node in edge_capacities:
                for edge in edge_capacities[node]:
                    if edge_capacities[node][edge] != 0:
                        augmented_graph_edges[node][edge] = edge_weights[node][edge]
                        
            # Print augmented capacities and augmentation paths
            if verbose:
                to_print = {}
                for node in edge_capacities:
                    to_print[node] = {}
                for node in edge_capacities:
                    for edge in edge_capacities[node]:
                        if edge_capacities[node][edge] != 0:
                            to_print[node][edge] = edge_capacities[node][edge]
                print('\nAugmented along: ' + str(augment_path) + ' with bottleneck capacity ' + str(bottleneck) + ' to yield the residual capacities:\n')
                for node in to_print:
                    print(node + ' : ' + str(to_print[node]))
                print('\n')
            
            # Get new augmentation path
            sp_graph.nodes = augmented_graph_edges

            if cycle_cancelling: # Get negative cost cycle
                augment_path = sp_graph.negative_cycles(begin, end, verbose = False)
                if augment_path is not None:
                    augment_path = augment_path[0]
            else: # get shortest path
                 augment_path = sp_graph.shortest_path(begin, end, verbose = False)
                
                
        return max_flow
    
    
def simplex(matrix, rhs, var_names):
    col_names = ['Z'] + var_names
    
    # Print Initial Tableau
    tableau = pd.DataFrame(matrix, columns = col_names)
    tableau['RHS'] = rhs
    row_names = ['OF']
    for i in range(matrix.shape[0]-1):
        row_names.append('E' + str(i+1))
    tableau.index = row_names
    
    print(tableau, '\n\n')
    
    
    # Termination Conditions
    if (tableau.iloc[0,1:-1] <= 0).all():
        print('Optimal')
        return
    
    for col in tableau.columns[1:-1]:
        if tableau[col][0] > 0 and (tableau[col][1:] <= 0).all():
            print('Unbounded')
            return
        
        
    pivot_column = tableau.iloc[0,1:-1].idxmax()
    pivot_row = (tableau['RHS'][1:]/tableau[pivot_column][1:]).idxmin()
    
    pivot_col_index = tableau.columns.tolist().index(pivot_column)
    pivot_row_index = tableau.index.tolist().index(pivot_row)
    
    new_tableau, new_rhs = _pivot(np.matrix(tableau.drop('RHS', 1)),
                                  np.array(tableau['RHS']), 
                                  pivot_row_index,
                                  pivot_col_index)
    
    print('Pivoted on ' + pivot_column + ', ' + pivot_row, '\n')
    
    simplex(new_tableau, new_rhs, var_names)

# Bellman-Ford Example

In [2]:
# Instantiate Graph
graph = Graph()

# Add Nodes to Graph
graph.add_weights('S', {'2': 1, '3': 4, '4': 5})
graph.add_weights('2', {'3': 2, 't': 4})
graph.add_weights('3', {'6': 4, 't': 1})
graph.add_weights('4', {'3': 6, '6': 0})
graph.add_weights('6', {'t': 6})
graph.add_weights('t', {})

# Run and Return Output
full_table, dst_table, path_table = graph.BellmanFord(begin = 'S', end = 't')

full_table

,init dst,init path,dst_1,path_1,dst_2,path_2,dst_3,path_3,dst_4,path_4
S,0,S,0,S,0.0,S,0.0,S,0.0,S
2,inf,unk,1,S,1.0,S,1.0,S,1.0,S
3,inf,unk,4,S,3.0,2,3.0,2,3.0,2
4,inf,unk,5,S,5.0,S,5.0,S,5.0,S
6,inf,unk,inf,unk,5.0,4,5.0,4,5.0,4
t,inf,unk,inf,unk,5.0,3,4.0,3,4.0,3


# Dijkstra Example

In [3]:
# Instantiate Graph
graph1 = Graph()

# Add Nodes to Graph
graph1.add_weights('S', {'A':1, 'C':2})
graph1.add_weights('A', {'B':6})
graph1.add_weights('B', {'E':2, 'D':1})
graph1.add_weights('C', {'A':4, 'D':3})
graph1.add_weights('D', {'E':1})
graph1.add_weights('E', {})

# Run and Return Output
full_table, dst_table, path_table = graph1.Dijkstra(begin = 'S', end = 'E')

full_table

,S_dst,S_path,A_dst,A_path,C_dst,C_path,D_dst,D_path,E_dst,E_path
S,0,S,0,S,0,S,0.0,S,0.0,S
A,1,S,1,S,1,S,1.0,S,1.0,S
B,inf,unk,7,A,7,A,7.0,A,7.0,A
C,2,S,2,S,2,S,2.0,S,2.0,S
D,inf,unk,inf,unk,5,C,5.0,C,5.0,C
E,inf,unk,inf,unk,inf,unk,6.0,D,6.0,D


# Dijkstra Bottleneck Shortest Path Example

### This will find the path with the minimum largest arc weight.

In [4]:
# Run and Return Output
full_table, dst_table, path_table = graph1.Dijkstra(begin = 'S', end = 'E', bottleneck = True)

dst_table

,S,A,C,D,E
S,0,0,0,0.0,0.0
A,1,1,1,1.0,1.0
B,inf,6,6,6.0,6.0
C,2,2,2,2.0,2.0
D,inf,inf,3,3.0,3.0
E,inf,inf,inf,3.0,3.0


# Shortest Path Example

In [5]:
# Instantiate Graph
graph2 = Graph()

# Add Nodes to Graph
graph2.add_weights('S', {'2':-1,'3':-4,'4':-5})
graph2.add_weights('2', {'t':-4})
graph2.add_weights('3', {'S':4,'2':2,'4':6,'t':-1})
graph2.add_weights('4', {'6':0})
graph2.add_weights('6', {'3':0,'4':0,'t':-6})
graph2.add_weights('t', {})

# Run and Print Output
graph2.shortest_path(begin = 'S', end = 't', method = 'BellmanFord')

('S', '4', '6', 't')

# Negative Cost Cycle Example

In [11]:
# Instantiate Graph
graph4 = Graph()

# Add Nodes to Graph
graph4.add_weights('S', {'A': 1, 'D':1, 't':8})
graph4.add_weights('A', {'B': -1})
graph4.add_weights('B', {'C': -1})
graph4.add_weights('C', {'A': -1, 't':1})
graph4.add_weights('D', {'E': -1})
graph4.add_weights('E', {'F': -1})
graph4.add_weights('F', {'D': -1, 't':1})
graph4.add_weights('t', {})

# Run and Print Output
graph5.negative_cycles(begin = 'S', end = 't')

(('A', 'B', 'C', 'A'), ('D', 'E', 'F', 'D'))

# Flow Augmentation Example

In [7]:
# Instantiate Graph
graph3 = Graph()

# Add weights to Graph
graph3.add_weights('S', {'a':0, 'c':0})
graph3.add_weights('a', {'b':1, 'c':4})
graph3.add_weights('b', {'c':2, 'd':7})
graph3.add_weights('c', {'d':3})
graph3.add_weights('d', {})

# Add upper bounds to Graph
graph3.add_bounds('S', {'a':2, 'c':3})
graph3.add_bounds('a', {'b':2, 'c':2})
graph3.add_bounds('b', {'c':3, 'd':4})
graph3.add_bounds('c', {'d':4})
graph3.add_bounds('d', {})

# Run and Print Output
graph3.flow_augmentation(begin = 'S', end = 'd', cycle_cancelling = False)


Augmented along: ('S', 'c', 'd') with bottleneck capacity 3 to yield the residual capacities:

S : {'a': 2}
a : {'b': 2, 'c': 2}
b : {'c': 3, 'd': 4}
c : {'S': 3, 'd': 1}
d : {'c': 3}



Augmented along: ('S', 'a', 'b', 'c', 'd') with bottleneck capacity 1 to yield the residual capacities:

S : {'a': 1}
a : {'S': 1, 'b': 1, 'c': 2}
b : {'a': 1, 'c': 2, 'd': 4}
c : {'S': 3, 'b': 1}
d : {'c': 4}



Augmented along: ('S', 'a', 'b', 'd') with bottleneck capacity 1 to yield the residual capacities:

S : {}
a : {'S': 2, 'c': 2}
b : {'a': 2, 'c': 2, 'd': 3}
c : {'S': 3, 'b': 1}
d : {'b': 1, 'c': 4}




5

# Negative Cost Cycle Cancelling Example

In [9]:
# Instantiate Graph
graph5 = Graph()

# Add weights to Graph
graph5.add_weights('S', {'A': 1, 'D':1, 't':8})
graph5.add_weights('A', {'B': -1})
graph5.add_weights('B', {'C': -1})
graph5.add_weights('C', {'A': -1, 't':1})
graph5.add_weights('D', {'E': -1})
graph5.add_weights('E', {'F': -1})
graph5.add_weights('F', {'D': -1, 't':1})
graph5.add_weights('t', {})

# Add upper bounds to Graph
graph5.add_bounds('S', {'A': 1, 'D':1, 't':1})
graph5.add_bounds('A', {'B': 1})
graph5.add_bounds('B', {'C': 1})
graph5.add_bounds('C', {'A': 1, 't':1})
graph5.add_bounds('D', {'E': 1})
graph5.add_bounds('E', {'F': 1})
graph5.add_bounds('F', {'D': 1, 't':1})
graph5.add_bounds('t', {})

# Run and Print Output
graph5.flow_augmentation(begin = 'S', end = 't', cycle_cancelling = True)


Augmented along: ('A', 'B', 'C', 'A') with bottleneck capacity 1 to yield the residual capacities:

S : {'A': 1, 'D': 1, 't': 1}
A : {'C': 1}
B : {'A': 1}
C : {'B': 1, 't': 1}
D : {'E': 1}
E : {'F': 1}
F : {'D': 1, 't': 1}
t : {}



Augmented along: ('D', 'E', 'F', 'D') with bottleneck capacity 1 to yield the residual capacities:

S : {'A': 1, 'D': 1, 't': 1}
A : {'C': 1}
B : {'A': 1}
C : {'B': 1, 't': 1}
D : {'F': 1}
E : {'D': 1}
F : {'E': 1, 't': 1}
t : {}




2

# Simplex Example

In [203]:
# your matrix including Z column and surplus variables
matrix = np.matrix([[1,3,5,0,0,0],
                    [0,1,0,1,0,0],
                    [0,0,2,0,1,0],
                    [0,3,2,0,0,1]])

# from top to bottom
rhs = np.array([0,8,24,36])

# variable names that will be in the tableau excluding 'Z' and 'RHS'
var_names = ['x1', 'x2', 'y1', 'y2', 'y3']

simplex(matrix, rhs, var_names)

    Z  x1  x2  y1  y2  y3  RHS
OF  1   3   5   0   0   0    0
E1  0   1   0   1   0   0    8
E2  0   0   2   0   1   0   24
E3  0   3   2   0   0   1   36 


Pivoted on x2, E2 

    Z  x1  x2  y1  y2  y3  RHS
OF  1   3   0   0   0   0  -60
E1  0   1   0   1   0   0    8
E2  0   0   1   0   0   0   12
E3  0   3   0   0   0   1   12 


Pivoted on x1, E3 

    Z  x1  x2  y1  y2  y3  RHS
OF  1   0   0   0   0   0  -72
E1  0   0   0   1   0   0    4
E2  0   0   1   0   0   0   12
E3  0   1   0   0   0   0    4 


Optimal
